In [ ]:
#Part three attempt 
from __future__ import absolute_import, division, print_function

#nomral os stuff 
import os
import pprint
import re
import logging
import codecs
import glob
import multiprocessing


In [278]:
#nltk is the best 
import nltk
#Couldnt get Glove working sooooo i tried ahah
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [279]:
data_filenames = sorted(glob.glob("query/queryall.trec"))

In [280]:
print("Found all the things:")
data_filenames

Found all the things:


['query/queryall.trec']

In [281]:
nltk.download("stopwords")


#formating or trying to anyway 
corpus_raw = u""
for data_filename in data_filenames:
    print("Reading '{0}'...".format(data_filename))
    with codecs.open(data_filename, "r", "utf-8") as data_file:
        corpus_raw += data_file.read()
    print("Corpus is now {0} characters long".format(len(corpus_raw)))
    print()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/butthead/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Reading 'query/queryall.trec'...
Corpus is now 12143 characters long



In [282]:
#nltk is life 
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [283]:
raw_data = tokenizer.tokenize(corpus_raw)

In [284]:
#Trying to remove all the shit 
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

In [285]:
sentences = []
for raw_sentence in raw_data:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

In [286]:
#kinda working, need to remove top, num etc
print(sentence_to_wordlist(raw_data[0]))

[u'top', u'num', u'num', u'title', u'AIRBUS', u'SUBSIDIES', u'title', u'top', u'top', u'num', u'num', u'title', u'SOUTH', u'AFRICAN', u'SANCTIONS', u'title', u'top', u'top', u'num', u'num', u'title', u'LEVERAGED', u'BUYOUTS', u'title', u'top', u'top', u'num', u'num', u'title', u'SATELLITE', u'LAUNCH', u'CONTRACTS', u'title', u'top', u'top', u'num', u'num', u'title', u'INSIDER', u'TRADING', u'title', u'top', u'top', u'num', u'num', u'title', u'PRIME', u'LENDING', u'RATE', u'MOVES', u'PREDICTIONS', u'title', u'top', u'top', u'num', u'num', u'title', u'MCI', u'title', u'top', u'top', u'num', u'num', u'title', u'RAIL', u'STRIKES', u'title', u'top', u'top', u'num', u'num', u'title', u'WEATHER', u'RELATED', u'FATALITIES', u'title', u'top', u'top', u'num', u'num', u'title', u'MERIT', u'PAY', u'VS', u'SENIORITY', u'title', u'top', u'top', u'num', u'num', u'title', u'ISRAELI', u'ROLE', u'IN', u'IRAN', u'CONTRA', u'AFFAIR', u'title', u'top', u'top', u'num', u'num', u'title', u'MILITARY', u'COUPS

In [287]:
token_count = sum([len(sentence) for sentence in sentences])
print("The data corpus contains {0:,} tokens of stuff".format(token_count))

The data corpus contains 1,690 tokens of stuff


In [288]:
#features can change for hopefully better results 
num_features = 300

# min word count 4 is my cut of could be 2 or 3 
min_word_count = 3
num_workers = multiprocessing.cpu_count()
#context size 
context_size = 10

downsampling = 1e-3
# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

In [293]:
data2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [234]:
#Build im so confused
data2vec.build_vocab(sentences)

In [235]:
#well thats wrong 
print("Word2Vec vocabulary length:", len(data2vec.wv.vocab))

Word2Vec vocabulary length: 38


In [245]:
data2vec.train(sentences, total_words=token_count, epochs=10)

(1925, 16900)

In [246]:
data2vec.save(os.path.join("trained", "data2.w2v"))

In [247]:
data2vec = glove2word2vec.Word2Vec.load(os.path.join("trained", "data2.w2v"))

In [305]:
print(data2vec.wv.vocab)

{}


In [248]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = data2vec.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{start1} is related to {end1}, as {start2} is related to {end2}".format(**locals()))
    return start2

In [254]:
data2vec.most_similar("TRADING")

/home/butthead/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[(u'FROM', 0.9997523427009583),
 (u'AGAINST', 0.999721884727478),
 (u'SATELLITE', 0.9997162818908691),
 (u'INTERNATIONAL', 0.9997023344039917),
 (u'INSIDER', 0.9996901154518127),
 (u'CONTROL', 0.9996888041496277),
 (u'ITS', 0.9996883273124695),
 (u'FOREIGN', 0.9996863603591919),
 (u'SYSTEM', 0.9996806383132935),
 (u'POLITICAL', 0.9996774792671204)]

In [256]:
# could be used for query expansion 
nearest_similarity_cosmul("COMPUTER", "CONTROL", "EQUIPMENT")

COMPUTER is related to CONTROL, as AIDED is related to EQUIPMENT


/home/butthead/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).


u'AIDED'

In [302]:
#Part four 
# need both query and raw_data
# raw_data = all the data 
# trying to get query id
# need the vectors
def get_doc_id(raw):
    clean = re.sub("[^1-9]"," ", raw)
    ids = clean.split()
    return ids

In [303]:
#please work 
def tfidf(raw_data, vectores)
    # inversed
    tfidf_model = models.TfidfModel(raw_data, id2word=gensim_dict)
    #comparisons
    tfidf_index = similarities.SparseMatrixSimilarity(
        tfidf_model[raw_data], num_features=len(gensim_dict.keys()))
    return tfidf_model, tfidf_index

In [ ]:
# define query expansion technique
def expand_query(query, embedding, k):
    # only expand terms in embedding vocab
    embedding_vocab = set(list(embedding.vocab.keys()))
    preprocess_query = query.lower().split()
    term_expansions = []

#hopefully be able to run this one day     
def trec_run(queries, rawdict, model, index, embedd):
    # run trec 
    run_results = dict()
    for (k, c) in rawdict.items():
        eq = expand_query(v, embedding, 10)  # expasion but a number
    return SortedDict(run_results)